# Generate CSV version of DISARM dataset

Generate CSV files to match older Excelfile versions of DISARM

In [ ]:

import pandas as pd
import os

MASTERDATA_DIR = '../DISARM_MASTER_DATA/'

    
frameworkfile = MASTERDATA_DIR + 'DISARM_FRAMEWORKS_MASTER.xlsx' 
datafile = MASTERDATA_DIR + 'DISARM_DATA_MASTER.xlsx'
#commentsfile = MASTERDATA_DIR + 'DISARM_COMMENTS_MASTER.csv'
        
# Get basic datasets from files
metadata = {}
for filein in [frameworkfile, datafile]:
    xlsx = pd.ExcelFile(filein)
    for sheetname in xlsx.sheet_names:
        print('{} sheet {}'.format(filein, sheetname))
        metadata[sheetname] = xlsx.parse(sheetname)
        metadata[sheetname].fillna('', inplace=True)
        metadata[sheetname].to_csv(MASTERDATA_DIR + sheetname + '.csv', index=False)

# create cross tables using DISARM code
def create_cross_table(df, col, newcol, divider=','):
    crosstable = df.join(df[col]
                    .str.split(divider, expand=True).stack()
                    .reset_index(drop=True,level=1)
                    .rename(newcol)).drop(col, axis=1)
    crosstable = crosstable[crosstable[newcol].notnull()]
    crosstable[newcol+'_id'] = crosstable[newcol].str.split(' ').str[0]
    crosstable.drop(newcol, axis=1, inplace=True)
    return crosstable

def create_incident_technique_crosstable(it_metadata):
    # Generate full cross-table between incidents and techniques
    it = it_metadata
    it.index=it['disarm_id']
    it = it['technique_ids'].str.split(',').apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('technique_id').reset_index().merge(it.drop('disarm_id', axis=1).reset_index()).drop('technique_ids', axis=1)
    it = it.merge(self.df_incidents[['disarm_id','name']], 
                  left_on='incident_id', right_on='disarm_id',
                  suffixes=['','_incident']).drop('incident_id', axis=1)
    it = it.merge(self.df_techniques[['disarm_id','name']], 
                  left_on='technique_id', right_on='disarm_id',
                  suffixes=['','_technique']).drop('technique_id', axis=1)
    return(it)

it = create_incident_technique_crosstable(metadata['incidenttechniques'])
cross_counterid_techniqueid = create_cross_table(df_counters[['disarm_id', 'techniques']], 
                                                           'techniques', 'technique', '\n')        
cross_counterid_resourceid = create_cross_table(df_counters[['disarm_id', 'resources_needed']], 
                                                          'resources_needed', 'resource', ',')
cross_counterid_actortypeid = create_cross_table(self.df_counters[['disarm_id', 'actortypes']], 
                                                          'actortypes', 'actortype', ',')
cross_detectionid_techniqueid = create_cross_table(self.df_detections[['disarm_id', 'techniques']], 
                                                           'techniques', 'technique', '\n')        
cross_detectionid_resourceid = create_cross_table(self.df_detections[['disarm_id', 'resources_needed']], 
                                                          'resources_needed', 'resource', ',')
cross_detectionid_actortypeid = create_cross_table(self.df_detections[['disarm_id', 'actortypes']], 
                                                          'actortypes', 'actortype', ',')

